In [ ]:
# !git clone https://github.com/giorgiodemarchi/ImageBind.git

In [ ]:
# !git clone https://github.com/giorgiodemarchi/Visually-Indicated-Sounds.git  ## This was done in a external colab notebook

In [11]:
# !pip install boto3

In [47]:
# !pip install --upgrade moviepy

In [4]:
!cd ImageBind; pip install .

Processing /workspace/Visually-Indicated-Sounds/ImageBind
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-b76qs7w1/pytorchvideo_de55ea27b5f24d3d9c46b7e94ce7a3c6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-b76qs7w1/pytorchvideo_de55ea27b5f24d3d9c46b7e94ce7a3c6
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 10.5 MB/s eta 0:00:00


In [48]:
from imagebind import data
import torch
import pandas as pd

from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

from imagebind.data import transform_and_sample_video_tensor

import numpy as np
from moviepy.editor import ImageSequenceClip

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [8]:
# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()

model.to(device)

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

**Create Dataset Instance and process video**

In [21]:
# Ontology audioset dataset
ontology = pd.read_json('data/ontology.json')[['id','name', 'description']]
augmented = pd.read_csv('data/augmented_labels.csv', index_col=0)

ontology = pd.merge(ontology, augmented, how='left', on='name')

ontology

,id,name,description,MajorityType,Object
0,/m/0dgw9r,Human sounds,Sounds produced by the human body through the ...,SFX,human body
1,/m/09l8g,Human voice,The human voice consists of sound made by a hu...,SFX,Human
2,/m/09x0r,Speech,Speech is the vocalized form of human communic...,SFX,Human
3,/m/05zppz,"Male speech, man speaking",Speech uttered by an adult male human.,SFX,An adult male human
4,/m/02zsn,"Female speech, woman speaking",Speech uttered by an adult female human.,SFX,woman
...,...,...,...,...,...
627,/m/025l19,Recording,A sound that appears to come from a recording ...,SFX,recorder/player
628,/m/07hvw1,Field recording,A sound that appears to have been recorded in ...,AMB,unknown
629,/m/0174nj,Gramophone record,A sound which appears to come from a gramophon...,SFX,gramophone or vinyl record disc on a turntable
630,/m/01www,Compact disc,A sound which appears to come from a digital a...,SFX,Compact Disc


In [22]:
#from VisuallyIndicatedSounds.utils.datasets import StronglyLabelledDataset
from utils.datasets import StronglyLabelledDataset

# Video-audio dataset on S3
dataset = StronglyLabelledDataset()

In [145]:
video, audio, labels_df, info = dataset[1600]

video_fps, audio_fps = info['video_fps'], info['audio_fps']

print(f"Video Shape: {video.shape}")
print(f"Audio Shape: {audio.shape}")

labels_df = pd.merge(labels_df, ontology, how='left', left_on='label', right_on='id')

labels_df

Video Shape: torch.Size([291, 360, 480, 3])
Audio Shape: torch.Size([2, 441344])


,segment_id,start_time_seconds,end_time_seconds,label,id,name,description,MajorityType,Object,similarity
0,2S2oAJ2XBYw_30000,0.0,10.0,/m/04229,/m/04229,Jet engine,The sound of an internal combustion airbreathi...,SFX,aircraft,2.089858e-08
1,2S2oAJ2XBYw_30000,0.0,10.0,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom",The sound made by the engine of a vehicle unde...,SFX,vehicle engine,2.763920e-09


In [146]:
video.shape

torch.Size([291, 360, 480, 3])

In [147]:
video_fps

29.065

In [118]:
# # Convert tensor to numpy array
# video_tensor = video # Assuming this is your video tensor

# if not torch.is_tensor(video_tensor):
#     video_tensor = torch.tensor(video_tensor)

# # Ensure it's in uint8, if not you might need to adjust the range and convert
# if video_tensor.dtype != torch.uint8:
#     video_tensor = (video_tensor.float() / video_tensor.max() * 255).byte()

# frames = video_tensor.numpy()

In [119]:
# video_clip = ImageSequenceClip(list(frames), fps=video_fps)

In [120]:
# video_clip

**Embed video**

In [148]:
## Working with the example video
## Required Transformations (Overwriting imagebind.data.load_and_transform_video)
scaled_video = transform_and_sample_video_tensor(video, device, clips_per_video=1)

print(f"Video Shape: {scaled_video.shape}")

inputs = {
    ModalityType.VISION: scaled_video,
}

with torch.no_grad():
    outputs = model(inputs)

print(f"Output Shape: {outputs['vision'].shape}")
video_embeddings = outputs['vision']

Video Shape: torch.Size([1, 291, 3, 224, 224])
Output Shape: torch.Size([1, 1024])


In [149]:
video_embeddings

tensor([[-0.0033, -0.0251,  0.0127,  ...,  0.0070, -0.0220,  0.0102]],
       device='cuda:0')

In [150]:
ontology.name.unique()

array(['Human sounds', 'Human voice', 'Speech',
       'Male speech, man speaking', 'Female speech, woman speaking',
       'Child speech, kid speaking', 'Conversation',
       'Narration, monologue', 'Babbling', 'Speech synthesizer', 'Shout',
       'Bellow', 'Whoop', 'Yell', 'Battle cry', 'Children shouting',
       'Screaming', 'Whispering', 'Laughter', 'Baby laughter', 'Giggle',
       'Snicker', 'Belly laugh', 'Chuckle, chortle', 'Crying, sobbing',
       'Baby cry, infant cry', 'Whimper', 'Wail, moan', 'Sigh', 'Singing',
       'Choir', 'Yodeling', 'Chant', 'Mantra', 'Male singing',
       'Female singing', 'Child singing', 'Synthetic singing', 'Rapping',
       'Humming', 'Groan', 'Grunt', 'Yawn', 'Whistling', 'Wolf-whistling',
       'Respiratory sounds', 'Breathing', 'Wheeze', 'Snoring', 'Gasp',
       'Pant', 'Snort', 'Cough', 'Throat clearing', 'Sneeze', 'Sniff',
       'Human locomotion', 'Run', 'Shuffle', 'Walk, footsteps',
       'Digestive', 'Chewing, mastication', 'Biti

**Embed Text audio labels**

In [151]:
labels_list = ontology.description.unique()

In [152]:
from imagebind import data

inputs = {
    ModalityType.TEXT: data.load_and_transform_text(labels_list, device)
}

In [153]:
with torch.no_grad():
    output = model(inputs)

text_embeddings = output[ModalityType.TEXT]

In [154]:
text_embeddings.shape

torch.Size([632, 1024])

In [155]:
vision_text_similarity = torch.softmax(video_embeddings @ text_embeddings.T, dim=-1)

In [156]:
ontology['similarity'] = vision_text_similarity.cpu().numpy()[0]

In [157]:
ontology.sort_values('similarity', ascending=False).head()

,id,name,description,MajorityType,Object,similarity
398,/m/014yck,Aircraft engine,The sound of the part of an aircraft that gene...,SFX,aircraft,0.970374
412,/t/dd00067,Heavy engine (low frequency),"The sound of a large, heavy-duty internal comb...",SFX,Internal combustion engine,0.010344
400,/m/02l6bg,"Propeller, airscrew",The sound of an aircraft propulsion system tha...,SFX,aircraft propulsion system,0.004773
407,/t/dd00065,Light engine (high frequency),"The sound of a small engine such as a toy car,...",SFX,moped,0.003851
399,/m/04229,Jet engine,The sound of an internal combustion airbreathi...,SFX,aircraft,0.001997


In [158]:
ontology[ontology['MajorityType']=='AMB'].sort_values('similarity', ascending=False).head()

,id,name,description,MajorityType,Object,similarity
481,/m/025wky1,Air conditioning,The sound of a device that removes heat from t...,AMB,"[{'name': 'Air conditioning', 'type': 'AMBIENC...",2.902888e-06
396,/m/0195fx,"Subway, metro, underground",Sounds of high-capacity electric railways that...,AMB,NaN,1.036542e-06
611,/m/06_y0by,Environmental noise,"The combined sounds of transport, industrial, ...",AMB,The object is undefined because the descriptio...,9.111949e-07
605,/t/dd00128,"Outside, urban or manmade",Sounds that appear to have been recorded outdo...,AMB,banana,1.978329e-07
614,/m/06xkwv,Mains hum,"A low, continuous tone caused by electromagnet...",AMB,fan,6.231134e-08


In [159]:
labels_df

,segment_id,start_time_seconds,end_time_seconds,label,id,name,description,MajorityType,Object,similarity
0,2S2oAJ2XBYw_30000,0.0,10.0,/m/04229,/m/04229,Jet engine,The sound of an internal combustion airbreathi...,SFX,aircraft,2.089858e-08
1,2S2oAJ2XBYw_30000,0.0,10.0,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom",The sound made by the engine of a vehicle unde...,SFX,vehicle engine,2.763920e-09


**Embed Audio**

In [21]:
audio.shape

torch.Size([2, 441344])

In [22]:
labels_df

,segment_id,start_time_seconds,end_time_seconds,label,id,name
0,3qDzHyrsWeg_30000,0.000,1.648,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"
1,3qDzHyrsWeg_30000,0.000,4.497,/m/03m9d0z,/m/03m9d0z,Wind
2,3qDzHyrsWeg_30000,0.000,4.497,/m/0838f,/m/0838f,Water
3,3qDzHyrsWeg_30000,0.000,4.511,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
4,3qDzHyrsWeg_30000,4.623,10.000,/m/02rlv9,/m/02rlv9,"Motorboat, speedboat"
5,3qDzHyrsWeg_30000,4.623,10.000,/m/03m9d0z,/m/03m9d0z,Wind
6,3qDzHyrsWeg_30000,4.623,10.000,/m/0838f,/m/0838f,Water
7,3qDzHyrsWeg_30000,4.623,10.000,/m/07q2z82,/m/07q2z82,"Accelerating, revving, vroom"


In [29]:
import torch
import torchaudio
from torchvision.transforms import Normalize
from pytorchvideo.data.clip_sampling import ConstantClipsPerVideoSampler

from imagebind.data import get_clip_timepoints, waveform2melspec

def load_and_transform_audio_data(
    audio_tensors,
    sample_rates,
    device,
    num_mel_bins=128,
    target_length=204,
    desired_sample_rate=16000,
    clip_duration=2,
    clips_per_video=3,
    mean=-4.268,
    std=9.138,
):
    if audio_tensors is None:
        return None

    audio_outputs = []
    clip_sampler = ConstantClipsPerVideoSampler(
        clip_duration=clip_duration, clips_per_video=clips_per_video
    )

    for waveform, sr in zip(audio_tensors, sample_rates):
        if desired_sample_rate != sr:
            waveform = torchaudio.functional.resample(
                waveform, orig_freq=sr, new_freq=desired_sample_rate
            )
        
        all_clips_timepoints = get_clip_timepoints(
            clip_sampler, waveform.size(1) / desired_sample_rate
        )
        all_clips = []
        for clip_timepoints in all_clips_timepoints:
            waveform_clip = waveform[
                :,
                int(clip_timepoints[0] * desired_sample_rate) : int(
                    clip_timepoints[1] * desired_sample_rate
                ),
            ]
            waveform_melspec = waveform2melspec(
                waveform_clip, desired_sample_rate, num_mel_bins, target_length
            )
            all_clips.append(waveform_melspec)

        normalize = Normalize(mean=mean, std=std)
        all_clips = [normalize(ac).to(device) for ac in all_clips]

        all_clips = torch.stack(all_clips, dim=0)
        audio_outputs.append(all_clips)

    return torch.stack(audio_outputs, dim=0)

In [35]:
# Read a bunch
audio_tensors = []
sample_rates = []

for index in range(1000,2000):
    _, audio, _, info = dataset[index]

    audio_tensors.append(audio)  # list of tensors
    sample_rates.append(audio_fps) # list of corresponding sample rates

KeyboardInterrupt: 

In [37]:
len(audio_tensors)

871

In [38]:
transformed_audio = load_and_transform_audio_data(
    audio_tensors, sample_rates, device='cuda'
)

In [44]:
transformed_audio = transformed_audio.cpu()

In [46]:
inputs = {
    ModalityType.AUDIO: transformed_audio.to(device)
}

with torch.no_grad():
    outputs = model(inputs)

audio_embeddings = outputs['audio']

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.15 GiB (GPU 0; 23.67 GiB total capacity; 21.37 GiB already allocated; 1.35 GiB free; 22.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
audio_embeddings.shape